In [97]:
import pandas as pd
import numpy as np
df = pd.read_csv("./datasets/Final_IC.csv")
df.head(5)

,article_id,article_desc
0,Article 1 of Indian Constitution,"Name and territory of the Union India, that is..."
1,Article 2 of Indian Constitution,Admission or establishment of new States: Parl...
2,Article 2A of Indian Constitution,Sikkim to be associated with the Union Rep by ...
3,Article 3 of Indian Constitution,Formation of new States and alteration of area...
4,Article 4 of Indian Constitution,Laws made under Articles 2 and 3 to provide fo...


In [98]:
df['ques'] = 'What is ' + df['article_id'].astype(str)
df['ans'] = df['article_id'].astype(str) + ' states that ' + df['article_desc']
df.drop('article_id',inplace=True,axis=1)
df.drop('article_desc',inplace=True,axis = 1)

## Conversion in format of BERT Pre-trained Model

In [99]:
data_list = []

for index, row in df.iterrows():
    data_dict = {
        'question': row['ques'],
        'context': row['ans']
    }
    data_list.append(data_dict)

In [100]:
data_list[343]

{'question': 'What is Article 292 of Indian Constitution',
 'context': 'Article 292 of Indian Constitution states that Borrowing by the Government of India The executive power of the Union extends to borrowing upon the security of the Consolidated Fund of India within such limits, if any, as may from time to time be fixed by Parliament by law and to the giving of guarantees within such limits, if any, as may be so fixed'}

## Importing the BERT Language Model

In [122]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer,Trainer, TrainingArguments
import torch
model_name = 'deepset/roberta-base-squad2'
model_bert = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer_bert = AutoTokenizer.from_pretrained(model_name)

Defining the inputs and outputs

In [102]:
input0 = tokenizer_bert(data_list[4]['question'],data_list[4]['context'],return_tensors='pt')
output0 = model_bert(**input0)

Detokenizing the answer using .decode()

In [103]:
answer_start_idx = torch.argmax(output0.start_logits)
answer_end_idx = torch.argmax(output0.end_logits)

answer_tokens = input0.input_ids[0,answer_start_idx:answer_end_idx + 1]
answer = tokenizer_bert.decode(answer_tokens)
print("Question: {}\nAnswer: {}".format(data_list[4]['question'],answer))

Question: What is Article 4 of Indian Constitution
Answer:  Laws made under Articles 2 and 3 to provide for the amendment of the First and the Fourth Schedules and supplemental, incidental and consequential matters


## Using LSTM

In [64]:
import tensorflow as tf
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences
Sequential = tf.keras.models.Sequential
Embedding = tf.keras.layers.Embedding
SimpleRNN = tf.keras.layers.SimpleRNN
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
GRU = tf.keras.layers.GRU
Dropout = tf.keras.layers.Dropout

In [58]:
questions = df['ques'].tolist()
answers = df['ans'].tolist()

In [119]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(answers)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(answers)[0]

# Prepare input and target sequences
input_sequences = []
output_sequences = []

sequence_length = 250
for i in range(len(sequences) - sequence_length):
    input_sequences.append(sequences[i:i + sequence_length])
    output_sequences.append(sequences[i + sequence_length])

input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

vocab_size = len(tokenizer.word_index) + 1

In [120]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=sequence_length),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(vocab_size, activation="softmax"),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 250, 32)           1408      
                                                                 
 lstm_13 (LSTM)              (None, 250, 128)          82432     
                                                                 
 lstm_14 (LSTM)              (None, 128)               131584    
                                                                 
 dense_14 (Dense)            (None, 44)                5676      
                                                                 
Total params: 221100 (863.67 KB)
Trainable params: 221100 (863.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [121]:
epochs = 100
batch_size = 32
model.fit(input_sequences, output_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/100
4/4 [==============================] - 6s 838ms/step - loss: 3.7708 - accuracy: 0.0469
Epoch 2/100
4/4 [==============================] - 4s 901ms/step - loss: 3.6471 - accuracy: 0.1406
Epoch 3/100
4/4 [==============================] - 4s 912ms/step - loss: 3.1013 - accuracy: 0.1328
Epoch 4/100
4/4 [==============================] - 4s 904ms/step - loss: 2.8521 - accuracy: 0.1094
Epoch 5/100
4/4 [==============================] - 4s 977ms/step - loss: 2.7752 - accuracy: 0.1562
Epoch 6/100
4/4 [==============================] - 5s 1s/step - loss: 2.7395 - accuracy: 0.1484
Epoch 7/100
4/4 [==============================] - 5s 1s/step - loss: 2.7231 - accuracy: 0.1328
Epoch 8/100
4/4 [==============================] - 5s 1s/step - loss: 2.7009 - accuracy: 0.1406
Epoch 9/100
4/4 [==============================] - 6s 1s/step - loss: 2.6954 - accuracy: 0.1719
Epoch 10/100
4/4 [==============================] - 6s 1s/step - loss: 2.6972 - accuracy: 0.1562
Epoch 11/100
4/4 [======

In [ ]:
#model.save("partially_trained.keras")

In [78]:
def generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate):
    generated_text = seed_text

    for _ in range(num_chars_to_generate):
        token_list = tokenizer.texts_to_sequences([generated_text])
        token_list = pad_sequences(token_list, maxlen=sequence_length, padding="pre")
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_token = np.argmax(predicted_probs, axis=-1)[0]  #

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_token:
                output_word = word
                break

        generated_text += output_word

    return generated_text

In [81]:
seed_text = "2"

generated_text = generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate=250)
print(generated_text)

2he theretes and the territories on the territories on the territories on the territories on the territories on the territories on the territories on the territories on the territories on the territories on the territories on the territories on the te
